# PSM to PeptideQ
Some useful functions:
* load psm data and sort it 
* load peptideQ data 
* join data function
* save file function
* controller function

In [1]:
import pandas as pd
import numpy as np
import os
import pyteomics.mzml
import spectrum_utils.spectrum as sus
from pathlib import Path

In [2]:
# We'll need to identify the matching columns
# we should not join the matching columns to the psm file 
# we should add the peptide data to the end of the psm file based on peptide sequence

In [70]:
# let's find the matching rows
psm_file_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\Ex_Auto_J3_30umTB_2ngQC_60m_1-calib_Peptides.psmtsv"

peptideQ_file_path = "C:\\Users\\Sarah Curtis\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\MetaM\\2ng\\AllQuantifiedPeptides.tsv"
psm_df = pd.read_table(psm_file_path, delimiter="\t")
peptideQ_df = pd.read_table(peptideQ_file_path, delimiter="\t")

duplicate_columns = []
for column in psm_df.columns:
    if column in peptideQ_df.columns:
        duplicate_columns.append(column)

duplicate_columns

['Base Sequence']

In [71]:
# looks like the only matching column is the 'Base Sequence' one so we don't need to worry about any of the other 
# columns

In [72]:
# read the psm file into a pandas dataframe
# sort dataframe by QValue
# drop duplicates

def load_psm(psm_file_path):
    # read the psm file into a pandas dataframe
    psm_dataframe = pd.read_table(psm_file_path, delimiter='\t')

    # sort dataframe by QValue
    psm_dataframe = psm_dataframe.sort_values("QValue")

    # drop duplicates
    psm_dataframe = psm_dataframe.drop_duplicates(subset=["Scan Number"], keep="first")
    psm_dataframe["Protein Accession"] = psm_dataframe["Protein Accession"].astype(str)

    # rename the "Full Sequence" column
    psm_dataframe = psm_dataframe.rename({"Full Sequence": "Peptide"})

    return psm_dataframe

def load_peptideQ(peptideQ_file_path):
    # read the peptideQ file into a pandas dataframe
    peptideQ_dataframe = pd.read_table(peptideQ_file_path, delimiter='\t')

    # rename the "Protein Groups" header so we can use the df.merge function later
    peptideQ_dataframe = peptideQ_dataframe.rename({"Protein Groups": "Protein Accession", "Sequence" : "Peptide"}, axis=1)
    
    return peptideQ_dataframe

def load_psm_df_msfragger(psm_file_path):
    # read in the psm file as a dataframe
    psm_df = pd.read_table(psm_file_path)

    # split the "Spectrum" column into a list at each period and store it under 
    # the "temp_split_column"
    psm_df["temp_split_column"] = psm_df["Spectrum"].str.split(".")
    # store the element located at index 1 of the "temp_split_column" in a 
    # "Scan Number" column
    psm_df["Scan Number"] = psm_df["temp_split_column"].map(lambda x:x[1]).apply(pd.to_numeric)
    # drop unneeded columns
    columns_to_drop = ['Spectrum', 'temp_split_column','Mapped Genes', 'Mapped Proteins']
    psm_df = psm_df.drop(columns=columns_to_drop)

    # rename 2 headers to match MM file formats
    psm_df = psm_df.rename({'Protein ID': 'Protein Accession', 'Spectrum File': 'File Name'}, axis=1)

    # drop duplicates
    psm_df = psm_df.drop_duplicates(subset=["Scan Number"], keep="first")
    
    return psm_df

def load_msfragger_peptideQ(peptideQ_file_path):
    # read the peptideQ file into a pandas dataframe
    peptideQ_dataframe = pd.read_table(peptideQ_file_path, delimiter='\t')

    # rename the "Protein Groups" header so we can use the df.merge function later
    peptideQ_dataframe = peptideQ_dataframe.rename({'Protein ID': 'Protein Accession', 'Peptide Sequence': 'Peptide'}, axis=1)
    
    return peptideQ_dataframe

In [73]:
# save the joined dataframe as a tsv file

def save_file(psm_dataframe, file_path):
    psm_dataframe.to_csv(file_path, sep="\t", index=False)
    print(f'Dataframe saved.')

In [74]:
def join_psm_and_peptideQ_dataframes(psm_df, peptideQ_df):
    # join based on the "Base Sequence"
    joined_dataframe = psm_df.merge(right=peptideQ_df, on="Base Sequence", how='inner')

    # generate multiIndex
    joined_dataframe = joined_dataframe.set_index(['File Name','Protein Accession','Peptide', 'Scan Number']).drop(columns=["Protein Groups"])

    return joined_dataframe


In [79]:
def psm_and_peptideQ_controller(file_type, psm_file_path, peptideQ_file_path, psm_and_peptideQ_file_path, columns_to_keep=None):
    
    ''' Joins a psm and a Peptide Quantification file. Files are joined into a pandas dataframe and saved as a tsv.
    
    Required Parameters:
        * file_type: "mm" for metamorpheus files, or "msfragger" for msfragger files
        * psm_file_path: File path to the psm file
        * peptideQ_file_path: File path to the Peptide Quantification file
        * psm_and_peptideQ_file_path: Output file path
        
    Optional Parameters: 
        * columns_to_keep: List of columns to include in the dataframe. Note that column names may vary based on whether your files were generated with MetaMorpheus or MSFragger.'''
        
    # load dataframes

    if file_type.lower() == 'mm':
        psm_df = load_psm(psm_file_path)
        peptideQ_df = load_peptideQ(peptideQ_file_path)
    elif file_type.lower() == 'msfragger':
        peptideQ_df = load_msfragger_peptideQ(peptideQ_file_path=peptideQ_file_path)
        psm_df = load_psm_df_msfragger(psm_file_path)
    else:
        print('invalid file type')
        return
    

    # join dataframes
    joined_df = join_psm_and_peptideQ_dataframes(psm_df, peptideQ_df)

    # select all columns to keep, if this parameter was not passed in, return dataframe with all columns
    if columns_to_keep != None:
        joined_df = joined_df[columns_to_keep]

    # save dataframe
    save_file(psm_dataframe=joined_df, file_path=psm_and_peptideQ_file_path)
    
    return joined_df

In [80]:
help(psm_and_peptideQ_controller)

Help on function psm_and_peptideQ_controller in module __main__:

psm_and_peptideQ_controller(file_type, psm_file_path, peptideQ_file_path, psm_and_peptideQ_file_path, columns_to_keep=None)
    Joins a psm and a Peptide Quantification file. Files are joined into a pandas dataframe and saved as a tsv.
    
    Required Parameters:
        * file_type: "mm" for metamorpheus files, or "msfragger" for msfragger files
        * psm_file_path: File path to the psm file
        * peptideQ_file_path: File path to the Peptide Quantification file
        * psm_and_peptideQ_file_path: Output file path
        
    Optional Parameters: 
        * columns_to_keep: List of columns to include in the dataframe. Note that column names may vary based on whether your files were generated with MetaMorpheus or MSFragger.



In [76]:
# testing the parser
outfile_path = "C:\\Users\\Sarah Curtis\\OneDrive - BYU\\Documents\\Single Cell Team Documents\\API_dev\\psm_and_peptideQ.tsv"
test_dataframe = controller(file_type='mm',psm_file_path=psm_file_path, peptideQ_file_path=peptideQ_file_path, psm_and_peptideQ_file_path=outfile_path)

KeyError: "None of ['Protein Accession', 'Peptide'] are in the columns"

In [64]:
test_dataframe.head(20)

Scan Retention Time  \
File Name                           Protein Accession Sequence                                           Scan Number                        
Ex_Auto_J3_30umTB_2ngQC_60m_1-calib P10809            LVQDVANNTNEEAGDGTTTATVLAR                          16668                   52.52059   
                                    P61106            IYQNIQDGSLDLNAAESGVQHK                             20217                   60.99236   
                                    P13639            ARPFPDGLAEDIDK                                     21843                   64.88158   
                                    Q00610            HSSLAGC[Common Fixed:Carbamidomethyl on C]QIINYR   16915                   53.10976   
                                    Q15365            LVVPATQC[Common Fixed:Carbamidomethyl on C]GSLIGK  19666                   59.67404   
                                    P13667            MDATANDVPSDR                                       8038                    31.98848   
                                    P07437            ISVYYNEATGGK                                       13743                   45.54150   
                                    P50395            DLGTESQIFISR                                       22397                   66.20470   
                                    P07737            SSFYVNGLTLGGQK                                     24022                   70.10585   
                                    Q16698            VAFITGGGTGLGK                                      19384                   58.99909   
                                    P12268            REDLVVAPAGITLK                                     22957                   67.54985   
                                    P30101            SEPIPESNDGPVK                                      10640                   38.15659   
                                    P78371            HGINC[Common Fixed:Carbamidomethyl on C]FINR       14597                   47.57920   
                                    P42771            ALLEAGALPNAPNSYGR                                  21273                   63.51660   
                                    P49321            VESLDVDSEAK                                        11248                   39.59846   
                                    Q15181            GISC[Common Fixed:Carbamidomethyl on C]M[Common... 23346                   68.47558   
                                                      GISC[Common Fixed:Carbamidomethyl on C]MNTTLSESPFK 23346                   68.47558   
                                                      GISC[Common Fixed:Carbamidomethyl on C]M[Common... 23378                   68.55237   
                                                      GISC[Common Fixed:Carbamidomethyl on C]MNTTLSESPFK 23378                   68.55237   
                                    P02545            AAYEAELGDAR                                        12204                   41.87444   

                                                                                                                      Num Experimental Peaks  \
File Name                           Protein Accession Sequence                                           Scan Number                           
Ex_Auto_J3_30umTB_2ngQC_60m_1-calib P10809            LVQDVANNTNEEAGDGTTTATVLAR                          16668                         101.0   
                                    P61106            IYQNIQDGSLDLNAAESGVQHK                             20217                          49.0   
                                    P13639            ARPFPDGLAEDIDK                                     21843                          77.0   
                                    Q00610            HSSLAGC[Common Fixed:Carbamidomethyl on C]QIINYR   16915                          99.0   
                                    Q15365            LVVPATQC[Common Fixed:Carbamidomethyl on C]GSLIGK  19666                          72.0   
       